In [1]:
import pandas as pd
import re
import csv
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
mpck = MPCK()

# Merge 전 각 DataFrame 정리:
 date, title, content, ngrams 컬럼, 날짜 형식 통일

## 금통위 의사록

In [5]:
#mp_report_ngram.csv 데이터프레임 정리:
mp=pd.read_csv("mp_report_ngram.csv", header=0, encoding='utf-8')
#컬럼명 바꾸기
mp = mp.rename(columns={'ngram': 'ngrams'})
#추가된 Unnamed index 컬럼 제거
mp.drop(mp.filter(regex="Unnamed: 0"),axis=1, inplace=True)
mp.drop(mp.filter(regex="Unnamed: 0.1"),axis=1, inplace=True)
# 금통위의사록 데이터프레임은 title 컬럼 없으므로 추가해줌
mp.insert(1,'title',None)
# 날짜 형식 yyyy.mm.dd로 변환
for i in range(len(mp)):
    mp['date'][i]=re.sub("-",".",mp['date'][i])
        
mp

,date,title,content,ngrams
0,2011-01-06,None,[],[]
1,2011-01-13,None,(['일부 위원은 최근 외국인 채권투자자·｢ ｣ 금의 순유출로 외환수급사정이 악화되...,"['외국인/NNG', '채권/NNG', '금/NNG', '순유출/NNG', '외환/..."
2,2011-01-20,None,[],[]
3,2011-02-11,None,(['일부 위원은 향후 미국의 통화정책 기조가·｢ ｣ 바뀔 경우 해외자금 유출입 흐...,"['통화정책/NNG', '기조/NNG', '바뀌/VV', '해외/NNG', '자금/..."
4,2011-02-24,None,[],[]
...,...,...,...,...
195,2020-09-24,None,[],[]
196,2020-10-14,None,(['일부 위원은 코로나19 사태에도 불구하고 중국 금융시장으로 외국인 증권투 자...,"['코로나/NNG', '사태/NNG', '불구/NNG', '금융시장/NNG', '외..."
197,2020-10-29,None,[],[]
198,2020-11-26,None,(['일부 위원은 글로벌 투자은행들이 내년에도 달러화의 약세 추세가 대체로 이 어...,"['글로벌/NNG', '투자은행/NNG', '달러/NNG', '약세/NNG', '추..."


## 채권리포트

In [ ]:
bond_report = pd.read_csv("bond_report_ultimate.csv", encoding = "utf-8")
#url 컬럼 제거
bond_report.drop(bond_report.filter(regex="url"),axis=1, inplace=True)
#tokens_ngrams 컬럼명 ngrams로 바꾸기
bond_report = bond_report.rename(columns={'tokens_ngrams': 'ngrams'})
bond_report

## 네이버 뉴스기사

In [ ]:
# 모든 뉴스기사 csv파일 합친 새 DataFrame 생성
DF_list = []
# ngram화 된 2011-2019년도 csv파일은 파일명이 규칙적이므로 for문 이용
for i in range(9):
    a = pd.read_csv(f'naver_ngram_201{i+1}.csv')
    DF_list.append(a)
DF_list.append(pd.read_csv('naver_ngram_2020.csv'))
#concat으로 합치기
Merged_na = pd.concat(DF_list, ignore_index=True)

# 컬럼명 한글이므로 영어로 바꿈
Merged_na = Merged_na.rename(columns={'날짜': 'date'})
Merged_na = Merged_na.rename(columns={'기사명': 'title'})
Merged_na = Merged_na.rename(columns={'본문': 'content'})
Merged_na = Merged_na.rename(columns={'tokens_ngrams': 'ngrams'})


In [ ]:
#네이버 뉴스 데이터프레임과 채권리포트 데이터프레임, mp 데이터프레임 merge
ngram_join = pd.concat([Merged_na, bond_report,mp], ignore_index=True)
ngram_join

## 콜금리

In [ ]:
call_index = pd.read_csv("call_index_.csv", encoding = "utf-8")
# 콜금리 데이터프레임에서 date,updown데이터만 가져오기
call_updown = pd.DataFrame(call_index, columns=["date","updown"])

In [ ]:
# 네이버뉴스,채권리포트 데이터프레임(ngram_join)과 콜금리 데이터프레임(call_updown) merge
ngram_call_join = pd.merge(call_updown,ngram_join, on='date',how='left')

In [ ]:
# 기존 날짜는 datetime type이 아니라 str형식이라 정렬 불가하므로 이 프로세스를 거쳐야함
# 뉴스,채권리포트,콜금리 merge된 데이터 프레임 내 datetime 타입으로 변환 및 정렬
import datetime

for i in range(len(ngram_call_join)):
    ngram_call_join['date'][i] = datetime.datetime.strptime(ngram_call_join['date'][i],'%Y.%m.%d')

ngram_call_join.sort_values(by=["date"], ascending=True)

# 2020년 이후의 콜금리 데이터 제거
ngram_call_join.drop([202051,202052],inplace=True)
ngram_call_join
